In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    # !wandb login
    # !pip install umap-learn
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/temporal')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Extra_and_Unused')
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "media/mirp_ai/Seagate Desktop Drive/Jaguas_2018"

Running local


In [2]:
# from __future__ import print_function
# from __future__ import print_function
import os
import numpy as np

import datetime
import gc

from scipy import signal

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform

from Jaguas_DataLoader_rainless import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel
from AE_Clustering import AE_Clustering 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()
print(device)

from datetime import timedelta
import wandb
from wandb import AlertLevel

# wandb.login()

cuda


In [3]:
# root_path = "ConservacionBiologicaIA/Datos/Jaguas_2018"
day = 27
hour = 19
date_format = f"day_{day}_hour_{hour}"
model_name = f"{root}/Jaguas/temporal/models/model_AE_batch_size_14_num_hiddens_64__{date_format}_final.pth"
config = torch.load(f'{root}/Jaguas/temporal/configs/config_AE_batch_size_14_num_hiddens_64__day_{day}_hour_{hour}.pth', map_location=torch.device('cpu'))
model = AE(num_hiddens=config["num_hiddens"]).to(device)
# dataset_test = torch.load(f'temporal/datasets/dataset_test_ae_jaguas_9_70%.pth')
# dataset_train = torch.load(f'temporal/datasets/dataset_train_ae_jaguas_9_70%.pth')
model.load_state_dict(torch.load(f'{model_name}', map_location=torch.device('cpu')))

<All keys matched successfully>

In [7]:
import os
folders = os.listdir(f"/{root_path}")


In [ ]:
# folders = ["Audios_Jaguas"]
for folder in folders:
    print(folder)
    filters = {"Intensity_Category": "No_rain"}
    dataset = SoundscapeData(root_path, dataframe_path=f"Complementary_Files/Audios_Jaguas/{folder[0:3]}.csv",
                             audio_length=12, ext="wav",
                             win_length=1028, filters=filters)
    featuring_autoencoders(dataset, date_format, model=model, len_features=None, save=True, identifier=f"{folder}")

G03_m
/media/mirp_ai/Seagate Desktop Drive/Jaguas_2018
id: 45 of 456
id: 90 of 456
id: 135 of 456
id: 180 of 456
id: 225 of 456
id: 270 of 456
id: 315 of 456
id: 360 of 456
id: 405 of 456
id: 450 of 456
G04_m
/media/mirp_ai/Seagate Desktop Drive/Jaguas_2018
id: 32 of 326
id: 64 of 326
id: 96 of 326
id: 128 of 326
id: 160 of 326
id: 192 of 326
id: 224 of 326
id: 256 of 326
id: 288 of 326
id: 320 of 326
G06_m
/media/mirp_ai/Seagate Desktop Drive/Jaguas_2018
id: 70 of 709
id: 140 of 709
id: 210 of 709
id: 280 of 709
id: 350 of 709
id: 420 of 709
id: 490 of 709
id: 560 of 709
id: 630 of 709
id: 700 of 709
G07_m
/media/mirp_ai/Seagate Desktop Drive/Jaguas_2018
id: 50 of 504
id: 100 of 504
id: 150 of 504
id: 200 of 504
id: 250 of 504
id: 300 of 504
id: 350 of 504
id: 400 of 504
id: 450 of 504
id: 500 of 504
G08_m
/media/mirp_ai/Seagate Desktop Drive/Jaguas_2018
id: 48 of 484
id: 96 of 484
id: 144 of 484
id: 192 of 484
id: 240 of 484
id: 288 of 484
id: 336 of 484
id: 384 of 484
id: 432 of 484

In [5]:
def featuring_autoencoders(dataset, date_format, model, len_features=None, save=True, identifier=None):
    training_loader = DataLoader(dataset, batch_size=1)
    # test_loader = DataLoader(dataset_test, batch_size=1)
    iterator = iter(training_loader)
    testing = TestModel(model, iterator, device=torch.device("cuda"))
    # encodings_size = encodings[0].shape

    training_recorder_list = []
    training_hour_list = []
    training_minute_list = []
    delete_samples = []
    training_path_samples = []
    training_samples_list_torch = []
    
    if len_features == None:
        len_features = len(training_loader)
    else:
        len_features = len_features
        
    batch = int(len_features*0.1)
    
    for id in range(len_features):
    #     if (id+1)%3 == 0:
    #         break
        if (id+1)% batch == 0:
            print(f"id: {id + 1} of {len_features}")
        try:
            originals, reconstructions, encodings, label, loss, path = testing.reconstruct()
        except:
            print(f"error id: {id}")
            delete_samples.append(id)
            continue

    #     encodings_size = encodings[0].shape
        encodings = encodings.to("cuda").detach()
        encodings = encodings.reshape(encodings.shape[0],
                                    encodings.shape[1]*encodings.shape[2]*encodings.shape[3])
        encoding = encodings.squeeze(dim=0)
        training_samples_list_torch.append(encodings)
        training_recorder_list.append(label["recorder"].reshape(label["recorder"].shape[0]*label["recorder"].shape[1]))
        training_hour_list.append(label["hour"].reshape(label["hour"].shape[0]*label["hour"].shape[1]))
        training_minute_list.append(label["minute"].reshape(label["minute"].shape[0]*label["minute"].shape[1]))


        path = np.asarray(path)
        path = np.repeat(path, 5)
        training_path_samples.append(path)

    training_recorder_list = torch.cat(training_recorder_list,dim=0)
    training_hour_list = torch.cat(training_hour_list,dim=0)
    training_minute_list = torch.cat(training_minute_list,dim=0)
    training_samples_list_torch = torch.cat(training_samples_list_torch, dim=0)
    
    if save == True:
    
        if "filters" in dataset.kwargs.keys():
            for value in dataset.filters.values():
                date_format = f"{date_format}_{value}" 

        if identifier != None:
            date_format = f"{date_format}_{identifier}"
        else:
            pass

        torch.save(training_path_samples, f"temporal/Features/test_path_samples_{date_format}.pth")
        torch.save(training_samples_list_torch, f"temporal/Features/test_samples_list_torch_{date_format}.pth")
        training_labels_list = {"recorder": training_recorder_list, "hour": training_hour_list, "minute": training_minute_list}
        torch.save(training_labels_list, f"temporal/Features/test_labels_list_{date_format}.pth")
        torch.save(delete_samples, f"temporal/Features/test_corrupted_samples_list_{date_format}.pth")
        
    else:
        pass

In [ ]:
training_loader = DataLoader(dataset, batch_size=1)
iterator = iter(training_loader)
testing = TestModel(model, iterator, device=torch.device("cuda"))
originals, reconstructions, encodings, label, loss, path = testing.reconstruct()
encodings[0].shape